In [1]:
# Ensemble Learning

# bootstrap sample
# 데이터 세트에서 중복을 허용하여 데이터를 샘플링하는 방식


# 랜덤 포레스트는 결정 트리를 랜덤하게 만들어 결정 트리(나무)의 숲을 만듭니다.
# 그리고 각 결정 트리의 예측을 사용해 최종 예측을 만듭니다
#
#
# bootstrap 방식으로 sample 추출
# 분류 모델인 RandomForestClassifier는 기본적으로 전체 특성 개수의 제곱근만큼의 특성을 무작위로 선택
# 다음 이 중에서 최선의 분할을 찾습니다

# 사이킷런에서는 100개 트


In [7]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs=-1, random_state=42)

# 평가할 모델 객체를 첫 번째 매개변수로 전달
# 그다음 앞에서처럼 직접 검증 세트를 떼어 내지 않고 훈련 세트 전체를 cross_validate() 함수에 전달
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

# 하나의 특성에 과도하게 집중하지 않고 좀 더 많은 특성이 훈련에 기여할 기회를 얻습니다.
# 이는 과대적합을 줄이고 일반화 성능을 높이는 데 도움이 됩니다

0.9973541965122431 0.8905151032797809


In [8]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


In [10]:
# RandomForestClassifier에는 자체적으로 모델을 평가하는 점수를 얻을 수 잇음
# Out of Bag Sample (OOB): bootstrap 안된 샘플
# oob 샘플을 사용해서 검증
rf = RandomForestClassifier(oob_score = True, n_jobs = -1, random_state=42)
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


In [13]:
# Extra Trees
# 100개의 결정 트리 훈련
# 랜덤포레스트와는 다르게 부트스트랩 샘플을 사용하지 않음
# 대신 노드 분할 시 가장 좋은 분할이 아닌 무작위 분할


from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target,
 return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

et.fit(train_input, train_target)
print(et.feature_importances_)

0.9974503966084433 0.8887848893166506
[0.20183568 0.52242907 0.27573525]


In [15]:
# Gradient Boosting
# 깊이가 얕은 결정 트리를 사용하여 이전 트리의 오차를 보완하는 방식으로 앙상블 하는 방법
# 사이킷런의 GradientBoostingClassifier는 기본적으로 깊이가 3인 결정 트리를 100개 사용


from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state = 42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [17]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2,
 random_state=42)
scores = cross_validate(gb, train_input, train_target,
 return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [19]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15872278 0.68010884 0.16116839]


In [20]:
# histogram gradient boosting
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target,
 return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

from sklearn.inspection import permutation_importance
hgb.fit(train_input, train_target)
# 특성 중요도를 계산
result = permutation_importance(hgb, train_input, train_target,
 n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)


/usr/local/lib/python3.10/dist-packages/sklearn/experimental/enable_hist_gradient_boosting.py:16: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


0.9321723946453317 0.8801241948619236
